In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
import xml.etree.ElementTree as et

In [2]:
html = requests.get('https://www.fedstat.ru/indicator/33459')
print(html.status_code)

200


In [3]:
soup = BeautifulSoup(html.text, "lxml")

In [4]:
#Pull the filters out of the html code
def get_filters(soup):
    script = soup.find_all("script")[11].text
    pattern = r"filters:\s*(\{.*?\})(?=\s*,\s*left_columns)"
    match = re.search(pattern, script, re.DOTALL).group(0)
    filters = "{" + match + "}"
    filters = re.sub(r'([{,]\s*)(\w+)(\s*:)', r'\1"\2"\3', filters)
    filters = filters.replace("'", '"')
    filters = json.loads(filters)
    return filters['filters']
filters = get_filters(soup)

In [5]:
# Find all available categories
def get_categories(filters):
    for key in filters.keys():
        print(filters[key]['title'])
get_categories(filters)

Показатель
Год
Возраст
Классификатор объектов административно-территориального деления (ОКАТО)
Единица измерения
Период
Тип поселения


In [6]:
#Find all available values for each category
categories =[]
for key in filters.keys():
    categories.append({
        key: list(filters[key]['values'].keys())
    })
ids = [f"{k}_{val}" for item in categories for k, v in item.items() for val in v]

# SDMX data

In [ ]:
#All values
data = {
        "lineObjectIds": ["0", "30611", "58335", "57831", "58274"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": ids
}
params = {
    "format" : "sdmx",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

In [7]:
# Adding  regions
regions = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="57831" for val in v]

In [8]:
#Adding years
years = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="3" for val in v]

In [9]:
#Adding settlement type
settlement_type = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="58274" for val in v]
settlement_type

['58274_1744150', '58274_1750789', '58274_1750788']

In [10]:
#Adding age categories
age =  [f"{k}_{val}" for item in categories for k, v in item.items() if k =="58335" for val in v]

In [12]:
#collect regional data
data = {
        "lineObjectIds": ["0", "30611", "58335", "57831", "58274"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": [
            "0_33459",
            years,
            "30611_950458",
            "33560_1540248",
            regions,
            "58274_1744150",
            "58335_1709566"
        ]
    }
params = {
    "format" : "sdmx",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

200 {'Content-Type': 'text/xml', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Sat, 12 Jul 2025 10:39:48 GMT', 'Set-Cookie': 'JSESSIONID=20E85FCD89FE3297FE03E7FAF9608248; Path=/; Secure; HttpOnly, session-cookie=18517ad4dc4c6a7edcd5a505204218c4d2506ea5bd73d1c7df057e3ecf38e193bc79ec9cdd66ebf6efc4768835a5ee4b; Max-Age=86400; Path=/; secure; HttpOnly', 'Cache-Control': 'no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': '0', 'Pragma': 'public', 'Content-Disposition': "attachment; filename*=UTF-8''data.xml", 'vary': 'accept-encoding', 'Content-Language': 'ru', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'Content-Security-Policy': 'upgrade-insecure-requests', 'Referrer-Policy': 'strict-origin'}


In [13]:
# Save the data in a file
with open("regions.xml", "wb") as file:
    file.write(response.content)

In [41]:
#Transform sdmx file into pandas dataframe
def read_sdmx(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    namespaces = {
        "generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic",
        "message": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/message",
        "structure": 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/structure'
    } 

    dataset = root.find(".//message:DataSet", namespaces)
    data = []
    for series in dataset.findall(".//generic:Series", namespaces):
        observations = series.findall(".//generic:Obs", namespaces)
        pair = {}
        for obs in observations:
            time = obs.find(".//generic:Time", namespaces).text
            value = obs.find(".//generic:ObsValue", namespaces).attrib['value']
            region = obs.find(".//")
            data.append({
                "year" : time,
                "population" : value
                })
    return pd.DataFrame(data)

data = read_sdmx("regions.xml")
data

,year,population
0,1990,78549601
1,1991,78817695
2,1992,78914788
3,1993,78899303
4,1994,78770458
...,...,...
3502,1996,3833075
3503,1999,3700204
3504,2000,3620686
3505,2001,3620686


# Excel data

In [11]:
#Let's get the data in excel
data = {
        "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": [
            regions,
            "0_33459",
            years,
            "30611_950458",
            "33560_1540248",
            settlement_type,
            age
        ]
    }
params = {
    "format" : "excel",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

200 {'Content-Type': 'application/vnd.ms-excel', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Sat, 12 Jul 2025 18:13:50 GMT', 'Set-Cookie': 'JSESSIONID=3C4282CC6A33C1401B08C39DC560B48E; Path=/; Secure; HttpOnly, session-cookie=185193194c22bedad248bc5a204218c49d8485e15add440b7691eb4b764627cc8c8430dea990c0f51427afc84e3cea45; Max-Age=86400; Path=/; secure; HttpOnly', 'Cache-Control': 'no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': '0', 'Pragma': 'public', 'Content-Disposition': "attachment; filename*=UTF-8''data.xls", 'Content-Language': 'ru', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'Content-Security-Policy': 'upgrade-insecure-requests', 'Referrer-Policy': 'strict-origin'}


In [ ]:
#Save the excel data
with open("all_data.xls", "wb") as file:
    file.write(response.content)

In [2]:
df = pd.read_excel("all_data.xls", header = 2)
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,1990,1991,1992,1993,1994,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,NaN,NaN,NaN,NaN,NaN,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,...,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января
1,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,все население,человек,78549601,78817695,78914788,78899303,78770458,...,78958558,79137859,79207619,79202094,79219390,78952485,78601633,78354712,78233436,78154709
2,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,городское население,человек,57983496,58303864,58267844,57878461,57688561,...,59279662,59512332,59660317,59755068,59858643,59691444,59487917,59352665,59312967,59399259
3,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,сельское население,человек,20566105,20513831,20646944,21020842,21081897,...,19678896,19625527,19547302,19447026,19360747,19261041,19113716,19002047,18920469,18755450
4,Численность постоянного населения - женщин по ...,Российская Федерация,0,все население,человек,1035959,957048,868672,770771,675810,...,942238,910777,814676,771071,714593,688397,673494,632575,611254,593327


In [3]:
df.drop([df.columns[0], df.columns[4]], axis = 1, inplace = True)
df = df.drop(0, axis = 0).reset_index(drop = True)

In [4]:
df.rename(columns = {
    'Unnamed: 1' : "region",
    "Unnamed: 2" : "age",
    "Unnamed: 3" : "settlement"
    }, inplace = True)

In [5]:
def get_min_age(age):
    numbers = re.findall(r'\d+', age)
    return int(numbers[0]) if numbers else None
    
def get_max_age(age):
    numbers = re.findall(r"\d+", age)
    return int(numbers[-1]) if numbers else None
    
def categorize_age(age_str):
    age_str = age_str.lower()
    numbers = re.findall(r'\d+', age_str)
    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', age_str, flags=re.IGNORECASE)
    if "-" in age_str and len(words) == 1:
        age_diff = int(numbers[1]) - int(numbers[0])
        return 2 if age_diff == 4 else 3
    elif "всего" in age_str or not numbers or len(words) > 3:
        return 4
    else:
        return 1

df['min_age']  = df['age'].apply(get_min_age)
df['max_age'] = df['age'].apply(get_max_age)
df['age_category'] = df['age'].apply(categorize_age)

In [6]:
df_sorted = df.groupby('region', sort = False).apply(
    lambda x: x.sort_values(['age_category', 'min_age', 'max_age'])
).reset_index(drop = True)

C:\Users\Denis\AppData\Local\Temp\ipykernel_6876\4101360515.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sorted = df.groupby('region', sort = False).apply(


In [9]:
df_sorted.head()

,region,age,settlement,1990,1991,1992,1993,1994,1995,1996,...,2019,2020,2021,2022,2023,2024,2025,min_age,max_age,age_category
0,Российская Федерация,0,все население,1035959,957048,868672,770771,675810,700146,670503,...,771071,714593,688397,673494,632575,611254,593327,0.0,0.0,1
1,Российская Федерация,0,городское население,731046,666347,594532,515664,451020,471582,454930,...,577689,538121,517804,506226,475039,458311,448261,0.0,0.0,1
2,Российская Федерация,0,сельское население,304913,290701,274140,255107,224790,228564,215573,...,193382,176472,170593,167268,157536,152943,145066,0.0,0.0,1
3,Российская Федерация,1 год,все население,1132245,1032202,961147,869881,773972,681638,700594,...,814838,771489,714665,689192,674423,633266,612805,1.0,1.0,1
4,Российская Федерация,1 год,городское население,799394,728256,666008,589740,516306,455062,472474,...,611994,579348,538639,519345,507480,476202,460383,1.0,1.0,1
